In [1]:
import gymnasium
import highway_env
# import finite-mdp
# from finite_mdp.envs import finite_mdp_env

%load_ext tensorboard
import sys
from tqdm.notebook import trange
# !pip install tensorboardx gym pyvirtualdisplay
# doesn't work cause not linux
# !apt-get install -y xvfb ffmpeg
# !git clone https://github.com/Farama-Foundation/HighwayEnv.git 2> /dev/null
# !git clone https://github.com/eleurent/finite-mdp.git 2> /dev/null
sys.path.insert(0, '/content/HighwayEnv/scripts/')
# from utils import record_videos, show_videos



In [2]:
# import gymnasium as gym
# import random
# import highway_env
# from collections import defaultdict

# import json
# import os


# # render_mode="rgb_array_list"
# env = gym.make("highway-fast-v0", render_mode = "rgb_array", config={
# #    "observation": {
# #     "type": "Kinematics",
# #     "vehicles_count": 5,
# #     "features": ["presence", "x", "y", "vx", "vy"],
# #     "features_range": {
# #         "x": [-100, 100],
# #         "y": [-100, 100],
# #         "vx": [-20, 20],
# #         "vy": [-20, 20]
# #     },
# #     "grid_size": [[-27.5, 27.5], [-27.5, 27.5]],
# #     "grid_step": [5, 5],
# #     "absolute": False,
# # },
# # Vehicle behaviour. defense vehicle
#  "observation": {
#     "type": "TimeToCollision",
#     "horizon": 2,
#     "lanes_count":2,
#     "other_vehicles_type": "highway_env.vehicle.behavior.DefenseVehicle",
#     # "lanes_count":2,
# },
# #   "action":{
# #     "type": "DiscreteAction",
# # }
# }
# )

# # obs, info = env.reset()


# class FiniteMarkovDecision():
#     def __init__(self,env):
#         self.finite_mdp = self.is_finite_mdp(env)
#         self.states = env.observation_space
#         if self.finite_mdp:
#             self.mpd = env.mdp
#         elif not self.finite_mdp:
#             try:
#                 self.mdp = env.unwrapped.to_finite_mdp()
#             except AttributeError:
#                 raise TypeError("not finite mdp")
            
#         self.env = env
#         self.obs, self.info = env.reset() 
#         self.state_action_value = env.action_space
#         self.load_dictionary()



#     def train(self):
#         gamma = 0.9  # Discount factor
#         episodes = 1000

#         # print(self.state_action_value.n)
#         for _ in range(episodes):  # Loop through each episode
#             done = False
#             total_reward = 0
#             episode_count = 0

#             # Initialize the environment and get the initial state
#             self.obs = self.env.reset()  # Assuming `reset` initializes the environment
#             state = str(self.obs)

#             while not done:
#                 # Select a random action
#                 action = random.randrange(0, self.state_action_value.n)

#                 # Take the action in the environment
#                 next_obs, reward, done, truncated, info = self.env.step(action)

#                 # Accumulate discounted rewards
#                 total_reward += reward * (gamma ** episode_count)
#                 episode_count += 1

#                 # Update policy if the current action is better
#             if state not in self.policy or self.policy[state]['reward'] < total_reward:
#                 self.policy[state] = {'action': action, 'reward': total_reward}

#                 # Update state for the next iteration
#                 # self.obs = next_obs
#                 # state = str(self.obs)
#             # env.render()
            

#             self.save_policy()


            
#     def load_dictionary(self):
#         file_path = "policies.json"
#         if os.path.exists(file_path):
#             try:
#                 with open(file_path, 'r') as file:  # Open in text mode for JSON
#                     self.policy = json.load(file)
#                     # print("Dictionary loaded successfully.")
#                     return True
#             except (json.JSONDecodeError, IOError) as e:
#                 print(f"Error loading dictionary: {e}")
#                 self.policy = {}  # Use a regular dictionary as fallback
#                 return False
#         else:
#             self.policy = {}  # Use a regular dictionary if file doesn't exist
#             print("No existing policy found. Starting with an empty dictionary.")
#             return False

#     def save_policy(self):
#         file_path = "policies.json"
#         try:
#             # Ensure self.policy is serializable
#             if isinstance(self.policy, defaultdict):
#                 self.policy = dict(self.policy)  # Convert defaultdict to dict

#             with open(file_path, 'w') as file:  # Open in write mode
#                 json.dump(self.policy, file, indent=4)  # Save with pretty printing
#                 # print("Dictionary saved successfully.")
#                 return True
#         except IOError as e:
#             print(f"Error saving dictionary: {e}")
#             return False
                
#     def evaluate(self, env, episode_num):
#         # add camera here
#         for _ in range(episode_num):
#             state = env.reset()[0]  
#             done = False      
#             truncated = False 
#             # Agent navigates map until it falls into a hole (terminated), reaches goal (terminated), or has taken 200 actions (truncated).
#             while(not done and not truncated):  
#                 # Select best action
#                 try:   
#                     action = self.policy[str(state)]
#                 except KeyError:
#                     action = random.randrange(0, env.action_space.n)
#                 next_state, reward, done, truncated, info = env.step(action)
#                 state = next_state
#                 env.render()



#     def is_finite_mdp(self,env):
#         try:
#             finite_mdp = __import__("finite_mdp.envs.finite_mdp_env")
#             if isinstance(env.unwrapped, finite_mdp.envs.finite_mdp_env.FiniteMDPEnv):
#                 return True
#         except (ModuleNotFoundError, TypeError):
#             return False



# finite_mdp = FiniteMarkovDecision(env)
# finite_mdp.train()
# finite_mdp.evaluate(env, 1)


In [4]:
import numpy as np
import json
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)
    
# https://stackoverflow.com/questions/50916422/python-typeerror-object-of-type-int64-is-not-json-serializable

In [ ]:

import gymnasium as gym
import json
import random
import highway_env
import numpy as np
from collections import defaultdict
import os
import sys
# import numpy

class QLearningAgent:
    def __init__(self, env, learning_rate=0.1, discount_factor=0.9, exploration_rate=1.0, exploration_decay=0.99):
        self.env = env
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay
        self.q_table = defaultdict()
        self.q_table_path = "q_table.json"
        self.load_q_table()
        self.action_space = env.action_space.n


    def choose_action(self, state):
        if random.random() < self.exploration_rate:
            return self.env.action_space.sample()  # Explore: random action
        else:
            try:
                return max(self.q_table[state])
            except:
                return self.env.action_space.sample()


    def train(self, episodes):
        for episode in range(episodes):
            state = str(self.env.reset()[0])  # Convert state to string for indexing
            done = False
            total_reward = 0

            while not done:
                action = str(self.choose_action(state))

                next_obs, reward, done, truncated, info = self.env.step(action)
                next_state = str(next_obs)


                if state not in self.q_table:
                    self.q_table[state] = { str(i): 0 for i in range(0, self.action_space)}


                if next_state not in self.q_table:
                    self.q_table[next_state] = { str(i): 0 for i in range(0, self.action_space)}


                best_next_action = str(max(self.q_table[next_state], key = self.q_table[next_state].get))
                # max(d, key = d.get)


                # try:
                #     self.q_table[next_state][best_next_action]
                # except:
                #     print(type(self.q_table[next_state].keys().get))

                
                td_target = reward + self.discount_factor * self.q_table[next_state][best_next_action]

                td_error = td_target - self.q_table[state][action]


                self.q_table[state][action] += self.learning_rate * td_error




                state = next_state
                total_reward += reward
                


            self.exploration_rate *= self.exploration_decay

            if (episode + 1) % 100 == 0:
                print(f"Episode {episode + 1}/{episodes}, Total Reward: {total_reward}")

        self.save_q_table()

    def evaluate(self, episodes=10):
        for episode in range(episodes):
            state = str(self.env.reset()[0])
            done = False
            total_reward = 0

            while not done:
                try:
                    action = max(self.q_table[state])
                except:
                    action = str(0)

                next_obs, reward, done, truncated, info = self.env.step(action)
                state = str(next_obs)
                total_reward += reward
                # self.env.render()

            print(f"Episode {episode + 1}/{episodes}, Total Reward: {total_reward}")

    def load_q_table(self):
        if os.path.exists(self.q_table_path):
            try:
                with open(self.q_table_path, 'r') as file:
                # self.q_table = np.load(self.q_table_path, allow_pickle=True).item()
                    loaded =  json.load(file)
                    self.q_table.update(loaded)
                    # print(type(self.q_table))
                    print("Q-table loaded successfully.")
            except Exception as e:
                print(f"Error loading Q-table: {e}")

    def save_q_table(self):
        try:
            with open(self.q_table_path, 'w') as file:  
                json.dump(self.q_table, file, indent=4)
                print("Q-table saved successfully.")
        except Exception as e:
            print(f"Error saving Q-table: {e}")


env = gym.make("highway-fast-v0", render_mode = "rgb_array", config={

 "observation": {
    "type": "TimeToCollision",
    "horizon": 5,
    "lanes_count":3,
    "other_vehicles_type": "highway_env.vehicle.behavior.DefenseVehicle",
    # "lanes_count":2,
},
}
)

# Initialize the Q-learning agent
agent = QLearningAgent(env)


# Train the agent
agent.train(episodes=10000)

# Evaluate the agent
# agent.evaluate(episodes=5)


Q-table loaded successfully.
